In [24]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# Creating Spark Session
sp = SparkSession.builder.appName("Customer").getOrCreate()

In [25]:
df_cc = sp.read.format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
    .option("dbtable", "CDW_SAPP_CREDIT_CARD") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

df_cust = sp.read.format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
    .option("dbtable", "CDW_SAPP_CUSTOMER") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

    

In [26]:
df_cc.show()

+-----------+----------------+---------+--------------+----------------+-----------------+--------+
|BRANCH_CODE|      CUST_CC_NO| CUST_SSN|TRANSACTION_ID|TRANSACTION_TYPE|TRANSACTION_VALUE|  TIMEID|
+-----------+----------------+---------+--------------+----------------+-----------------+--------+
|        156|4210653312478046|123455692|         22562|         Grocery|            91.08|20180813|
|        180|4210653342242023|123451310|         45069|           Bills|            77.79|20180315|
|        114|4210653349028689|123459988|             1|       Education|             78.9|20180214|
|         35|4210653349028689|123459988|             2|   Entertainment|            14.24|20180320|
|        107|4210653342242023|123451310|         45070|      Healthcare|            20.47|20180419|
|        114|4210653312478046|123455692|         22563|           Bills|             22.2|20180626|
|        160|4210653349028689|123459988|             3|         Grocery|             56.7|20180708|


In [27]:
df_join = df_cc.join(df_cust, df_cc.CUST_SSN == df_cust.SSN, 'inner').select(col('SSN'), col('CUST_ZIP'), col('TIMEID'), \
      col('TRANSACTION_ID'), col('TRANSACTION_VALUE'))


In [28]:
df_join.show()

+---------+--------+--------+--------------+-----------------+
|      SSN|CUST_ZIP|  TIMEID|TRANSACTION_ID|TRANSACTION_VALUE|
+---------+--------+--------+--------------+-----------------+
|123455659|   10954|20180723|         22772|            97.57|
|123455659|   10954|20180625|         22773|            65.32|
|123455659|   10954|20181022|         22774|             5.76|
|123455659|   10954|20180521|         22775|           100.81|
|123455659|   10954|20180726|         22776|            57.93|
|123455659|   10954|20180513|         22777|            76.18|
|123455659|   10954|20181119|         22778|            87.72|
|123455659|   10954|20180228|         22779|            25.73|
|123455659|   10954|20181002|         22780|            58.56|
|123455659|   10954|20180105|         22781|             75.2|
|123455659|   10954|20181207|         22782|            70.19|
|123455659|   10954|20180827|         22783|            85.66|
|123455659|   10954|20180908|         22784|           

In [29]:
df_join.createOrReplaceTempView('CC_CUST')

In [15]:
data = sp.sql('select * from CC_CUST where CUST_ZIP == "53066" and substr(TIMEID,1,4) == "2018" and substr(TIMEID,5,2)=="06"')
data.show(100)

+---------+-----------------+--------+--------+
|      SSN|TRANSACTION_VALUE|  TIMEID|CUST_ZIP|
+---------+-----------------+--------+--------+
|123451357|            18.13|20180606|   53066|
|123451357|            13.92|20180506|   53066|
|123451357|            35.08|20180706|   53066|
|123451357|            65.42|20181106|   53066|
|123451357|            18.13|20180606|   53066|
|123451357|            13.92|20180506|   53066|
|123451357|            35.08|20180706|   53066|
|123451357|            65.42|20181106|   53066|
|123454391|            57.18|20180306|   53066|
|123454391|             3.71|20180706|   53066|
|123454391|             23.4|20180306|   53066|
|123454391|            64.11|20180306|   53066|
|123454391|            96.96|20180806|   53066|
|123454391|            57.18|20180306|   53066|
|123454391|             3.71|20180706|   53066|
|123454391|             23.4|20180306|   53066|
|123454391|            64.11|20180306|   53066|
|123454391|            96.96|20180806|  

2. Functional Requirements - Application Front-End
 2.1 Transaction Details Module

In [30]:
# 1. Used to display the transactions made by customers living in a given zip code for a given month and year. 
# Order by day in descending order.

def trans_value(zip,yr,mm):
    print("inside the function", zip, yr, mm)
    data = sp.sql('select distinct * from CC_CUST where CUST_ZIP == "{}" and substr(TIMEID,1,4) == "{}" \
        and substr(TIMEID,5,2) == "{}" ORDER BY substr(TIMEID,7,2) DESC'.format(zip,yr,mm))
    data.show()


zip = str(input("enter zipcode: "))
yr = str(input("enter year: "))
mm = str(input("enter month: "))
trans_value(zip,yr,mm)


inside the function 53066 2018 06
+---------+--------+--------+--------------+-----------------+
|      SSN|CUST_ZIP|  TIMEID|TRANSACTION_ID|TRANSACTION_VALUE|
+---------+--------+--------+--------------+-----------------+
|123458614|   53066|20180628|          6691|             19.2|
|123457562|   53066|20180628|         12436|            65.84|
|123457286|   53066|20180627|         13783|            42.01|
|123451357|   53066|20180626|         44896|            13.47|
|123458614|   53066|20180625|          6697|             6.66|
|123457286|   53066|20180623|         13824|            52.27|
|123454357|   53066|20180622|         29654|            82.03|
|123457286|   53066|20180622|         13799|            41.65|
|123457562|   53066|20180621|         12400|            17.85|
|123457562|   53066|20180621|         12387|            21.05|
|123454391|   53066|20180620|         29420|             82.4|
|123452342|   53066|20180618|         40354|            38.26|
|123451357|   53066|2

In [ ]:
# 2. Used to display the number and total values of transactions for a given type.



In [ ]:
data_br = df_cc.join(df_br, df_br.BRANCH_CODE == df_cc.BRANCH_CODE, 'inner').select(col('df_cc.branch_code'), \
            col('df_br.branch_name'), col('df_cc.transaction_value'))
data_br.show()


In [2]:
import pymysql
import pandas as pd
import matplotlib.pyplot as plt
import pandas.io.sql as psql
import seaborn as sns

In [15]:
# connect to Database

connection = pymysql.connect(
    host='localhost',
    user='root',
    password='password',
    database='creditcard_capstone'
)

In [20]:
# fetch data from table to dataframe

cursor = connection.cursor()
sql = "SELECT * FROM CDW_SAPP_loan_application"
cursor.execute(sql)
result = cursor.fetchall()
df_loan = psql.read_sql(sql, con=connection)
pd.set_option("display.max_columns", None)

# print(df_loan .head())
df_loan.shape

C:\Users\Learner_XZHCG217\AppData\Local\Temp\ipykernel_15276\770319098.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_loan = psql.read_sql(sql, con=connection)


(1022, 10)

In [22]:
# 5.1 Find and plot the percentage of applications approved for self-employed applicants.

df_loan_plot = df_loan.groupby(['Self_Employed','Application_Status'])['Application_ID'].count()
df_loan_plot.head()

# df_loan_plot = df_loan_plot.reset_index()
# total_applications = len(data_frame)
# #data_api = data_api[(data_api['Application_Status'] == 'Y') & (data_api['Self_Employed'] == 'Yes' ) ]
# data_api['Count_Percent'] = data_api['Application_ID']/total_applications * 100
# data_api
# sns.set(rc={"figure.figsize":(6,5)})
# sns.set_theme(style="whitegrid", palette="pastel")
# sns.barplot(x ='Self_Employed',
#             y = 'Count_Percent',
#             #hue='Application_Status',
#             hue='Count_Percent',
#             data = data_api).set(title ="Percentage of applications approved for self-employed applicants." )

Self_Employed  Application_Status
No             N                     280
               Y                     602
Yes            N                      48
               Y                      92
Name: Application_ID, dtype: int64

In [ ]:
# 5.2 Find the percentage of rejection for married male applicants.

